In [1]:
import io
import js2py
import base64
import pandas as pd

In [2]:
df = pd.read_fwf(io.StringIO("""2021-09-10T00:55:04Z home-water       04b6481955196490 water-meter-01 1      EWEBAAA=
2021-09-11T00:54:32Z home-water       04b6481955196490 water-meter-01 1      EQUCAAA=
2021-09-12T00:54:52Z home-water       04b6481955196490 water-meter-01 1      EaACAAA=
2021-09-13T00:55:06Z home-water       04b6481955196490 water-meter-01 1      EWADAAA=
"""), names=["time", "app", "eui", "name", "port", "payload"])
df["time"] = pd.to_datetime(df["time"])

In [3]:
df

,time,app,eui,name,port,payload
0,2021-09-10 00:55:04+00:00,home-water,04b6481955196490,water-meter-01,1,EWEBAAA=
1,2021-09-11 00:54:32+00:00,home-water,04b6481955196490,water-meter-01,1,EQUCAAA=
2,2021-09-12 00:54:52+00:00,home-water,04b6481955196490,water-meter-01,1,EaACAAA=
3,2021-09-13 00:55:06+00:00,home-water,04b6481955196490,water-meter-01,1,EWADAAA=


In [4]:
def index(s):
    b = base64.b64decode(s)
    print(b)
    return (b[1] | (b[2] << 8) | (b[3] << 16) | (b[4] << 24)) / 1000.

In [5]:
df["index"] = df["payload"].apply(index)

b'\x11a\x01\x00\x00'
b'\x11\x05\x02\x00\x00'
b'\x11\xa0\x02\x00\x00'
b'\x11`\x03\x00\x00'


In [6]:
df

,time,app,eui,name,port,payload,index
0,2021-09-10 00:55:04+00:00,home-water,04b6481955196490,water-meter-01,1,EWEBAAA=,0.353
1,2021-09-11 00:54:32+00:00,home-water,04b6481955196490,water-meter-01,1,EQUCAAA=,0.517
2,2021-09-12 00:54:52+00:00,home-water,04b6481955196490,water-meter-01,1,EaACAAA=,0.672
3,2021-09-13 00:55:06+00:00,home-water,04b6481955196490,water-meter-01,1,EWADAAA=,0.864


In [7]:
bin(0xf0)

'0b11110000'

In [8]:
code = """
function Decode(port, bytes, variables) {
  var data = {
    "data": bytes,
    "type": bytes[0] & 0x0f,
    "subtype": (bytes[0] & 0xf0) >> 4
  };
  if (data["type"] == 1) {
    data["index"] = (bytes[1] | (bytes[2] << 8) | (bytes[3] << 16) | (bytes[4] << 24)) / 1000.
  }
  return data;
}
var x = Decode(2, %s, null);
"""

In [9]:
def decode(s):
    b = [int(c) for c in base64.b64decode(s)]
    return js2py.eval_js(code % b)

In [10]:
df["decoded"] = df["payload"].apply(decode)

In [11]:
df["decoded"].to_list()

[{'data': [17, 97, 1, 0, 0], 'index': 0.353, 'subtype': 1, 'type': 1},
 {'data': [17, 5, 2, 0, 0], 'index': 0.517, 'subtype': 1, 'type': 1},
 {'data': [17, 160, 2, 0, 0], 'index': 0.672, 'subtype': 1, 'type': 1},
 {'data': [17, 96, 3, 0, 0], 'index': 0.864, 'subtype': 1, 'type': 1}]

In [46]:
df["stamp"] = df["time"].apply(lambda x: int(x.timestamp()))*1000000000

In [47]:
df

,time,app,eui,name,port,payload,index,decoded,stamp
0,2021-09-10 00:55:04+00:00,home-water,04b6481955196490,water-meter-01,1,EWEBAAA=,0.353,"{'data': [17, 97, 1, 0, 0], 'index': 0.353, 's...",1631235304000000000
1,2021-09-11 00:54:32+00:00,home-water,04b6481955196490,water-meter-01,1,EQUCAAA=,0.517,"{'data': [17, 5, 2, 0, 0], 'index': 0.517, 'su...",1631321672000000000
2,2021-09-12 00:54:52+00:00,home-water,04b6481955196490,water-meter-01,1,EaACAAA=,0.672,"{'data': [17, 160, 2, 0, 0], 'index': 0.672, '...",1631408092000000000
3,2021-09-13 00:55:06+00:00,home-water,04b6481955196490,water-meter-01,1,EWADAAA=,0.864,"{'data': [17, 96, 3, 0, 0], 'index': 0.864, 's...",1631494506000000000


In [48]:
for r in df.to_dict(orient="records"):
    print("INSERT device_frmpayload_data_index,application_name={app:},dev_eui={eui:},device_name={name:},f_port={port:} value={index:} {stamp:}".format(**r))

INSERT device_frmpayload_data_index,application_name=home-water,dev_eui=04b6481955196490,device_name=water-meter-01,f_port=1 value=0.353 1631235304000000000
INSERT device_frmpayload_data_index,application_name=home-water,dev_eui=04b6481955196490,device_name=water-meter-01,f_port=1 value=0.517 1631321672000000000
INSERT device_frmpayload_data_index,application_name=home-water,dev_eui=04b6481955196490,device_name=water-meter-01,f_port=1 value=0.672 1631408092000000000
INSERT device_frmpayload_data_index,application_name=home-water,dev_eui=04b6481955196490,device_name=water-meter-01,f_port=1 value=0.864 1631494506000000000
